# Simple illustration of DMRG for fermionic Hubbard model
This notebook adapts from this paper

    Eric Jeckelmann, Density-Matrix Renormalization Group Algorithms, 2008

as well as the implementation from

https://simple-dmrg.readthedocs.io/en/latest/

https://dmrg101-tutorial.readthedocs.io/en/latest/index.html

https://github.com/Andrew-Shen/DMRG/tree/master/DMRG-Hubbard

to solve the one-dimensional Hubbard model

$$H=-t\sum_{i=1}^{L-1} \sum_{\sigma} \left(c_{i,\sigma}^{\dagger} c_{i+1,\sigma} + h.c. \right) + \sum_{i=1}^L U n_{i,\uparrow} n_{i,\downarrow} - \mu (n_{i,\uparrow}+n_{i,\downarrow}).$$

The implementation below is compatible with Julia 1.0. It uses chemical potential to control the total number of electrons.

This is very similar to the Heisenberg XXZ model after the Jordan-Wigner transformation. The only difference is that one needs to carefully distinguish the left and right sides of the blocks.



<br>

Lin Lin

Last revision: 9/21/2018
<br><br>


Some convention used in the code:

- Each site has dimension 4, with the convention following the Kronecker product: spin up $\otimes$ spin down, i.e. the spin down occupies the inner dimension (row-major)

- For creation operators $c_{i,\sigma}^\dagger$, larger $i$ occupies the inner dimension (i.e. row-major, consistent with Kronecker product)

- The Jordan-Wigner transformation is

$$c_{i,\sigma} = \sigma_z\otimes\cdots \otimes \sigma_z \otimes A_{\sigma} \otimes I\otimes \cdots\otimes I,$$

i.e. the pairity accumulates towards the left direction. This means the convention for creating a determinant is e.g. $c_{1,\sigma}^{\dagger}c_{2,\sigma}^{\dagger}c_{3,\sigma}^{\dagger}\vert 0\rangle$.

After the JW transformation, the 1D Hubbard model becomes again a local Hamiltonian

$$H=-\sum_{i=1}^{N-1} \sum_{\sigma} \left[(a_{i,\sigma}^{\dagger} \sigma^{z,\uparrow,\downarrow}_{i}) a_{i+1,\sigma} + (\sigma^{z,\uparrow,\downarrow}_{i} a_{i,\sigma}) a_{i+1,\sigma}^{\dagger}  \right]+  \sum_{i=1}^L U n_{i,\uparrow} n_{i,\downarrow} - \mu (n_{i,\uparrow}+n_{i,\downarrow}) n_{i,\downarrow}.$$

Here 

$$a_{i,\sigma} =  I\otimes \cdots\otimes I \otimes A_{\sigma} \otimes I\otimes \cdots\otimes I$$
$$a^{\dagger}_{i,\sigma} =  I\otimes \cdots\otimes I \otimes A^{\dagger}_{\sigma} \otimes I\otimes \cdots\otimes I$$
$$\sigma^{z,\uparrow,\downarrow}_{i} = I\otimes \cdots\otimes I \otimes \sigma^{z,\uparrow,\downarrow} \otimes I\otimes \cdots\otimes I$$

are the bosonic annihilation and creation operator and commute (hence note the reversed order in the term $(\sigma^{z,\uparrow,\downarrow}_{i} a_{i,\sigma}) a_{i+1,\sigma}^{\dagger}$. This is OK since operators on different sites now commute), while the definition of $n_{i,\sigma}$ are the same.


In the setup of the Hamiltonian, the Coulomb term acts only on one site, and the main complexity of the codes goes to the treatment of the hopping term involving two sites. To simplify the treatment, the sign operators $\sigma^{z,\uparrow,\downarrow}$ from the Jordan-Wigner transformation is always combined with $a_{i,\sigma}$ (called the "connecting operators") in the code. Note that this distinguishes the treatment of the left and right blocks from the beginning.
 
`enlarge_block` increases the system by one site from $j$ to $j+1$. Hence the tensor product of the the previous Hamiltonian with the identity is needed, plus the new connecting part of the Hamiltonian.  

The infinite DMRG algorithm (at least the infinite version) contains the following steps. 

1. Construct the superblock Hamiltonian via the tensor product of the previously compressed components.  This is not a memory-efficient implementation since it does not use the low-rank structure of the superblock Hamiltonian.

2. The update of the MPS basis is always on the left, by performing an eigenvalue decomposition of the reduced density matrix on the new site on the left. The update on the right block mirrors the update on the left.


The finite DMRG algorithm first performs a "warmup" sweeping step using the infinite DMRG algorithm to reach site L. Then the blocks are stored on a "disk" (in this case memory).  In the consequent sweeping steps, the finite system DMRG first sweeps from left to right, and then right to left. The use of the name `sys` and `env` rather than `left` and `right` is mainly for the finite system DMRG, where `sys` and `env` alternatively assumes the role of `left` and `right` in different sweeping steps. 

Unlike the infinite-system algorithm, the `env` part always reads the MPS basis from the "disk" (i.e. the valuesfrom the previous sweep), and correspondingly the "disk" is updated by the `sys` part.


In [1]:
using Arpack
using LinearAlgebra
using SparseArrays

# The length of the EnlargedBlock is always the length of the
# corresponding Block plus 1
struct Block
    length::Int
    basis_size::Int
    operator_dict::Dict{Symbol,AbstractMatrix{Float64}}
end

struct EnlargedBlock
    length::Int
    basis_size::Int
    operator_dict::Dict{Symbol,AbstractMatrix{Float64}}
end

# Model-specific code for the 1D Hubbard model
model_d = 4  # single-site basis size

I1 = Matrix(1.0I,2,2)
Z1 = [1.0 0.0; 0.0 -1.0]
N1 = [0.0 0.0; 0.0 1.0]
A1 = [0.0 1.0; 0.0 0.0]  # single qubit annihilation
A1_dag = A1'
c_up1     = kron(A1,I1)
c_dag_up1 = kron(A1_dag,I1)
c_dn1     = kron(Z1,A1)
c_dag_dn1 = kron(Z1,A1_dag)

n_up1 = kron(N1,I1)
n_dn1 = kron(I1,N1)
I_updn1 = kron(I1,I1)
Z_updn1 = kron(Z1,Z1)
n_updn1 = n_up1 * n_dn1 # only the (4,4) entry is 1

t = -1.0
U = 0.5
mu = 0.0

H1 = U * n_updn1 - mu * (n_up1 + n_dn1)



# Note that in the Kronecker structure, L always appears on the left,
# even for terms such as c^{\dagger}_R c_L. The anti-commutation sign
# has been taken care of
function H2(c_up_L, c_dag_up_L, c_dn_L, c_dag_dn_L, 
            c_up_R, c_dag_up_R, c_dn_R, c_dag_dn_R)  
    return t * (kron(c_dag_up_L, c_up_R) + kron(c_up_L, c_dag_up_R) + 
                kron(c_dag_dn_L, c_dn_R) + kron(c_dn_L, c_dag_dn_R) )
end

# conn refers to the connection operator, that is, the operator on the edge of
# the block, on the interior of the chain. 
#
# In Block, conn_* operators are represented in the current MPS basis.
#
# In EnlargedBlock, conn_* are simply the tensor product of * operators
# with proper identity tensors.
#
# Due to the Jordan-Wigner transformation, the initial block on the left
# and right are different.
#
initial_block_L = Block(1, model_d, 
    Dict{Symbol,AbstractMatrix{Float64}}(
    :H => H1,
    :conn_c_up     => Z_updn1 * c_up1,
    :conn_c_dag_up => c_dag_up1 * Z_updn1,
    :conn_c_dn     => Z_updn1 * c_dn1,
    :conn_c_dag_dn => c_dag_dn1 * Z_updn1)
)

initial_block_R = Block(1, model_d, 
    Dict{Symbol,AbstractMatrix{Float64}}(
    :H => H1,
    :conn_c_up     => c_up1,
    :conn_c_dag_up => c_dag_up1,
    :conn_c_dn     => c_dn1,
    :conn_c_dag_dn => c_dag_dn1)
)

function enlarge_block(block::Block, block_label::Symbol)
    # This function enlarges the provided Block by a single site, returning an
    # EnlargedBlock.
    #
    # This way how tensor product is performed depends on whether the
    # block is on the left or on the right, following a consistent
    # convention of Kronecker products.
    mblock = block.basis_size
    o = block.operator_dict

    # Create the new operators for the enlarged block.  
    I_site = sparse(1.0I, model_d, model_d)
    I_block = sparse(1.0I, mblock, mblock)
    if( block_label == :l )
        newH = kron(o[:H], I_site) + kron(I_block, H1) +
            H2(o[:conn_c_up], o[:conn_c_dag_up], o[:conn_c_dn],
               o[:conn_c_dag_dn], c_up1, c_dag_up1, c_dn1, c_dag_dn1)
        enlarged_operator_dict = Dict{Symbol,AbstractMatrix{Float64}}(
            :H => newH,
            :conn_c_up     => kron(I_block, Z_updn1 * c_up1),
            :conn_c_dag_up => kron(I_block, c_dag_up1 * Z_updn1),
            :conn_c_dn     => kron(I_block, Z_updn1 * c_dn1),
            :conn_c_dag_dn => kron(I_block, c_dag_dn1 * Z_updn1)
           )

    elseif( block_label == :r )
        newH = kron(I_site, o[:H]) + kron(H1, I_block) +
            H2(Z_updn1 * c_up1, c_dag_up1 * Z_updn1, Z_updn1 * c_dn1, c_dag_dn1 * Z_updn1, 
               o[:conn_c_up], o[:conn_c_dag_up], o[:conn_c_dn], o[:conn_c_dag_dn])
        enlarged_operator_dict = Dict{Symbol,AbstractMatrix{Float64}}(
            :H => newH,
            :conn_c_up     => kron(c_up1, I_block),
            :conn_c_dag_up => kron(c_dag_up1, I_block),
            :conn_c_dn     => kron(c_dn1, I_block),
            :conn_c_dag_dn => kron(c_dag_dn1, I_block))
    else
        error("Wrong label for enlarge_block.")
    end
    
    return EnlargedBlock(block.length + 1,
                         block.basis_size * model_d,
                         enlarged_operator_dict)
end


function solve_super_hamiltonian(blocks_enl::Dict{Symbol,EnlargedBlock})
    # Construct the full superblock Hamiltonian corresponding to the two
    # sites in the middle.
    m_blockL_enl = blocks_enl[:l].basis_size
    m_blockR_enl = blocks_enl[:r].basis_size
    I_blockL_enl = sparse(1.0I, m_blockL_enl, m_blockL_enl)
    I_blockR_enl = sparse(1.0I, m_blockR_enl, m_blockR_enl)

    H_L             = blocks_enl[:l].operator_dict[:H]
    conn_c_up_L     = blocks_enl[:l].operator_dict[:conn_c_up]
    conn_c_dag_up_L = blocks_enl[:l].operator_dict[:conn_c_dag_up]
    conn_c_dn_L     = blocks_enl[:l].operator_dict[:conn_c_dn]
    conn_c_dag_dn_L = blocks_enl[:l].operator_dict[:conn_c_dag_dn]

    H_R             = blocks_enl[:r].operator_dict[:H]
    conn_c_up_R     = blocks_enl[:r].operator_dict[:conn_c_up]
    conn_c_dag_up_R = blocks_enl[:r].operator_dict[:conn_c_dag_up]
    conn_c_dn_R     = blocks_enl[:r].operator_dict[:conn_c_dn]
    conn_c_dag_dn_R = blocks_enl[:r].operator_dict[:conn_c_dag_dn]

    superblock_hamiltonian = kron(H_L, I_blockR_enl) +
        kron(I_blockL_enl, H_R) +
        H2(conn_c_up_L, conn_c_dag_up_L, conn_c_dn_L, conn_c_dag_dn_L,
           conn_c_up_R, conn_c_dag_up_R, conn_c_dn_R, conn_c_dag_dn_R) 
    superblock_hamiltonian = (superblock_hamiltonian +
                              superblock_hamiltonian') / 2.0

    # Call ARPACK to find the superblock ground state.  (:SR means find the
    # eigenvalue with the "smallest real" value.)
    (energy,), psi0 = eigs(superblock_hamiltonian, nev=1, which=:SR)
    return (energy, psi0)
end

function update_block(blocks_enl::Dict{Symbol,EnlargedBlock}, psi0,
                      m::Int, sys_label::Symbol)
    # Construct the reduced density matrix of the system by tracing out
    # the environment. 
    #
    # (sys_label, env_label) should be (:l,:r) or (:r,:l), respectively.
    #

    m_blockL_enl = blocks_enl[:l].basis_size
    m_blockR_enl = blocks_enl[:r].basis_size


    # rotate the system part
    
    # We want to make the (sys, env) indices correspond to (row, column) of a
    # matrix, respectively.  Since the environment (column) index updates most
    # quickly in our Kronecker product structure, psi0 is thus row-major.
    # However, Julia stores matrices in column-major format, so we first
    # construct our matrix in (env, sys) form and then take the transpose.
    psi0_t = transpose(reshape(psi0, (m_blockR_enl, m_blockL_enl)))

    fact = svd(psi0_t)
    evals = fact.S.^2
    if( sys_label == :l )
        evecs = fact.U
    else
        evecs = fact.V
    end

    # Build the transformation matrix from the `m` overall most significant
    # eigenvectors.
    my_m = min(length(evals), m)
    indices = 1:my_m
    basis = evecs[:, indices]

    truncation_error = 1 - sum(evals[indices])
    println("truncation error: ", truncation_error)

    # Rotate and truncate each operator.
    new_operator_dict = Dict{Symbol,AbstractMatrix{Float64}}()
    for (name, op) in blocks_enl[sys_label].operator_dict
        new_operator_dict[name] = basis' * (op * basis)
    end

    newblock = Block(blocks_enl[sys_label].length, my_m, new_operator_dict)
    return newblock
end



function infinite_dmrg_step(blocks::Dict{Symbol,Block}, m::Int)
    # Performs a single infinite DMRG step, and keeping a maximum of `m`
    # states in the new basis.
    #
    # Infinite DMRG algorithm uses the same psi0 to update the system
    # and environment part.


    # Enlarge each block by a single site.
    blocks_enl = Dict{Symbol,EnlargedBlock}()
    
    blocks_enl[:l] = enlarge_block(blocks[:l],:l)
    blocks_enl[:r] = enlarge_block(blocks[:r],:r)

    (energy, psi0) = solve_super_hamiltonian( blocks_enl )
    
    newblocks = Dict{Symbol,Block}()
    newblocks[:l] = update_block( blocks_enl, psi0, m, :l )
    newblocks[:r] = update_block( blocks_enl, psi0, m, :r )

    return newblocks, energy
end

function finite_dmrg_step(blocks::Dict{Symbol,Block}, m::Int,
                          sys_label::Symbol)
    # Performs a single finite DMRG step, and keeping a maximum of `m`
    # states in the new basis.
    #
    # Finite DMRG algorithm only updates the system part, and keeps the
    # environment part intact.

    # Enlarge each block by a single site.
    blocks_enl = Dict{Symbol,EnlargedBlock}()
    
    blocks_enl[:l] = enlarge_block(blocks[:l],:l)
    blocks_enl[:r] = enlarge_block(blocks[:r],:r)

    (energy, psi0) = solve_super_hamiltonian( blocks_enl )

    # Only update the system part
    newblocks = copy( blocks )
    newblocks[sys_label] = update_block( blocks_enl, psi0, m, sys_label )

    return newblocks, energy
end

function graphic(blocks::Dict{Symbol,Block}, sys_label::Symbol)
    # Returns a graphical representation of the DMRG step we are about to
    # perform, using '=' to represent the system sites, '-' to represent the
    # environment sites, and '**' to represent the two intermediate sites.

    if( sys_label == :l )
        str = repeat("=", blocks[:l].length) * "**" * repeat("-", blocks[:r].length)
    elseif( sys_label == :r )
        str = repeat("-", blocks[:l].length) * "**" * repeat("=", blocks[:r].length)
    else
        error("Wrong label.")
    end

    return str
end




function infinite_system_algorithm(L::Int, m::Int)
    # Repeatedly enlarge the system by performing a single DMRG step, using a
    # reflection of the current block as the environment.
    blocks = Dict{Symbol,Block}()
    blocks[:l] = initial_block_L
    blocks[:r] = initial_block_R
    println("Infinite-system DMRG sweeping to with bond dimension m = ", m)

    while blocks[:l].length + blocks[:r].length < L
        println(graphic(blocks, :l))
        current_L = blocks[:l].length + blocks[:r].length + 2
        println("L = ", current_L)
        blocks, energy = infinite_dmrg_step(blocks, m)
        println("E/L = ", energy / current_L )
    end
end


function finite_system_algorithm(L::Int, m_warmup::Int, m_sweep_list::AbstractVector{Int})
    @assert iseven(L)

    # To keep things simple, this dictionary is not actually saved to disk, but
    # we use it to represent persistent storage.
    block_disk = Dict{Tuple{Symbol,Int},Block}()  # "disk" storage for Block objects

    # Use the infinite system algorithm to build up to desired size.  Each time
    # we construct a block, we save it for future reference as both a left
    # (:l) and right (:r) block, as the infinite system algorithm assumes the
    # environment is a mirror image of the system.
    blocks = Dict{Symbol,Block}()
    blocks[:l] = initial_block_L
    blocks[:r] = initial_block_R
    
    block_disk[:l, blocks[:l].length] = blocks[:l]
    block_disk[:r, blocks[:r].length] = blocks[:r]
    println("Infinite-system DMRG sweeping to warm up with bond dimension m = ", m_warmup)
    while blocks[:l].length + blocks[:r].length < L
        # Perform a single DMRG step and save the new Block to "disk"
        println(graphic(blocks, :l))
        current_L = blocks[:l].length + blocks[:r].length + 2
        blocks, energy = infinite_dmrg_step(blocks, m_warmup)
        println("E/L = ", energy / current_L )
        block_disk[:l, blocks[:l].length] = blocks[:l]
        block_disk[:r, blocks[:r].length] = blocks[:r]
    end

    # Now that the system is built up to its full size, we perform sweeps using
    # the finite system algorithm.  At first the left block will act as the
    # system, growing at the expense of the right block (the environment), but
    # once we come to the end of the chain these roles will be reversed.
    sys_label, env_label = :l, :r

    for m in m_sweep_list
        println()
        println("Finite-system DMRG with bond dimension m = ", m)
        while true
            # Load the appropriate environment block from "disk"
            blocks[env_label] = block_disk[env_label, L - blocks[sys_label].length - 2]
            if blocks[env_label].length == 1
                # We've come to the end of the chain, so we reverse course.
                sys_label, env_label = env_label, sys_label
            end

            # Perform a single DMRG step.
            println(graphic(blocks, sys_label))
            blocks, energy = finite_dmrg_step(blocks, m, sys_label)

            println("E/L = ", energy / L)
            println("E   = ", energy)

            # Save the block from this step to disk.
            block_disk[sys_label, blocks[sys_label].length] = blocks[sys_label]

            # Check whether we just completed a full sweep.
            if sys_label == :l && (2 * blocks[sys_label].length == L)
                break  # escape from the "while true" loop
            end
        end
    end
end

finite_system_algorithm (generic function with 1 method)

In the following, the graphical form `====**----` indicates the progress of the DMRG algorithms. Here `=` means the system sites, `-` means the environment sites, and `**` means the two intermediate sites.

In [2]:
println("Infinite system algorithm.")
infinite_system_algorithm(10, 10)

Infinite system algorithm.
Infinite-system DMRG sweeping to with bond dimension m = 10
=**-
L = 4
truncation error: 0.0021992215724561115
truncation error: 0.0021992215724561115
E/L = -0.9995209608521296
==**--
L = 6
truncation error: 0.003017842261356929
truncation error: 0.003017842261356929
E/L = -1.0411732670900633
===**---
L = 8
truncation error: 0.0030809949775668644
truncation error: 0.0030809949775668644
E/L = -1.0626739331216792
====**----
L = 10
truncation error: 0.004700199946294004
truncation error: 0.004700199946294004
E/L = -1.0757676316292077


Reference energy for this system (t=-1, U=0.5, mu=0)

E = -10.8564601187

In [3]:
println("Finite system algorithm.")
finite_system_algorithm(10, 10, [10,10,20,20,30,30])


Finite system algorithm.
Infinite-system DMRG sweeping to warm up with bond dimension m = 10
=**-
truncation error: 0.0021992215724554454
truncation error: 0.0021992215724554454
E/L = -0.9995209608521309
==**--
truncation error: 0.0029564880734924692
truncation error: 0.0029564880734924692
E/L = -1.0410202173380372
===**---
truncation error: 0.0031994100170787965
truncation error: 0.0031994100170787965
E/L = -1.062594610829812
====**----
truncation error: 0.00468576135001153
truncation error: 0.00468576135001153
E/L = -1.0756867538423938

Finite-system DMRG with bond dimension m = 10
=====**---
truncation error: 0.003132896075047409
E/L = -1.0755134446349466
E   = -10.755134446349466
======**--
truncation error: 0.003818711578915557
E/L = -1.075596375758672
E   = -10.75596375758672
-------**=
truncation error: 0.0024547084118672347
E/L = -1.0755555811725963
E   = -10.755555811725962
------**==
truncation error: 0.0038605597596687025
E/L = -1.075766798347836
E   = -10.75766798347836
---